<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Space X  Falcon 9 First Stage Landing Prediction**


## Web scraping Falcon 9 and Falcon Heavy Launches Records from Wikipedia


Estimated time needed: **40** minutes


In this lab, you will be performing web scraping to collect Falcon 9 historical launch records from a Wikipedia page titled `List of Falcon 9 and Falcon Heavy launches`

https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/Falcon9_rocket_family.svg)


Falcon 9 first stage will land successfully


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/landing_1.gif)


Several examples of an unsuccessful landing are shown here:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/crash.gif)


More specifically, the launch records are stored in a HTML table shown below:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/falcon9-launches-wiki.png)


  ## Objectives
Web scrap Falcon 9 launch records with `BeautifulSoup`: 
- Extract a Falcon 9 launch records HTML table from Wikipedia
- Parse the table and convert it into a Pandas data frame


First let's import required packages for this lab


In [59]:
!pip3 install beautifulsoup4
!pip3 install requests

In [60]:
import sys

import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd

and we will provide some helper functions for you to process web scraped HTML table


In [61]:
def date_time(table_cells):
    """
    This function returns the data and time from the HTML  table cell
    Input: the  element of a table data cell extracts extra row
    """
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    """
    This function returns the booster version from the HTML  table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=''.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])
    return out

def landing_status(table_cells):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=[i for i in table_cells.strings][0]
    return out


def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass


def extract_column_from_header(row):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
        
    colunm_name = ' '.join(row.contents)
    
    # Filter the digit and empty names
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name    


To keep the lab tasks consistent, you will be asked to scrape the data from a snapshot of the  `List of Falcon 9 and Falcon Heavy launches` Wikipage updated on
`9th June 2021`


In [63]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

Next, request the HTML page from the above URL and get a `response` object


### TASK 1: Request the Falcon9 Launch Wiki page from its URL


First, let's perform an HTTP GET method to request the Falcon9 Launch HTML page, as an HTTP response.


In [7]:
# use requests.get() method with the provided static_url
# assign the response to a object
# 1. Make the HTTP GET request
response = requests.get("https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922")

# 2. Create BeautifulSoup object
soup = BeautifulSoup(response.text, 'html.parser')

# 3. Verify success (optional)
print(soup.title.text)  # Should show the Wikipedia page title

List of Falcon 9 and Falcon Heavy launches - Wikipedia


Create a `BeautifulSoup` object from the HTML `response`


In [9]:
# Use BeautifulSoup() to create a BeautifulSoup object from a response text content
# 1. Make the HTTP request (if not already done)
response = requests.get("https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922")

# 2. Create BeautifulSoup object with lxml parser (faster than default)
soup = BeautifulSoup(response.content, 'lxml')  # or use 'html.parser' if lxml not available

# 3. Verify (optional)
print(f"Page title: {soup.title.string}")  # Should show "List of Falcon 9 and Falcon Heavy launches"

Page title: List of Falcon 9 and Falcon Heavy launches - Wikipedia


Print the page title to verify if the `BeautifulSoup` object was created properly 


In [10]:
# Use soup.title attribute
# Print the page title to confirm successful parsing
print("Page title:", soup.title.text)

Page title: List of Falcon 9 and Falcon Heavy launches - Wikipedia


### TASK 2: Extract all column/variable names from the HTML table header


Next, we want to collect all relevant column names from the HTML table header


Let's try to find all tables on the wiki page first. If you need to refresh your memory about `BeautifulSoup`, please check the external reference link towards the end of this lab


In [12]:
# Use the find_all function in the BeautifulSoup object, with element type `table`
# Assign the result to a list called `html_tables`
# 1. Find all HTML tables (returns a list)
html_tables = soup.find_all('table', {'class': 'wikitable'})

# 2. Select the correct table (3rd table in this case)
launch_table = html_tables[2]  # Index 2 for 3rd table

# 3. Extract column names from table headers
column_names = []
for header in launch_table.find_all('th'):
    # Use provided helper function to clean header text
    col_name = extract_column_from_header(header)
    if col_name:  # Only keep non-empty names
        column_names.append(col_name)

# 4. Verify extracted columns
print("Extracted Columns:", column_names)

Extracted Columns: ['Flight No.', 'Date and time ( )', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome']


Starting from the third table is our target table contains the actual launch records.


In [13]:
# Let's print the third table and check its content
first_launch_table = html_tables[2]
print(first_launch_table)

<table class="wikitable plainrowheaders collapsible" style="width: 100%;">
<tbody><tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date andtime ()
</th>
<th scope="col">
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launchoutcome
</th>
<th scope="col">
</th></tr>
<tr>
<th rowspan="2" scope="row" style="text-align:center;">14
</th>
<td>10 January 2015,<br/>09:47<sup class="reference" id="cite_ref-nasa20150107_74-0"><a href="#cite_note-nasa20150107-74"><span class="cite-bracket">[</span>67<span class="cite-bracket">]</span></a></sup>
</td>
<td><a href="/wiki/Falcon_9_v1.1" title="Falcon 9 v1.1">F9 v1.1</a><br/>B1012<sup class="reference" id="cite_ref-block_numbers_14-8"><a href="#cite_note-block_numbers-14"><span class="cite-bracket">[</span>8<span class="cite-bracket">]</span></a></sup>
</td>
<td><a href="/wiki/Cape_Canaveral_Space_Force_Station" tit

You should able to see the columns names embedded in the table header elements `<th>` as follows:


```
<tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11">[b]</a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12">[c]</a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 9 first-stage landing tests">Booster<br/>landing</a>
</th></tr>
```


Next, we just need to iterate through the `<th>` elements and apply the provided `extract_column_from_header()` to extract column name one by one


In [19]:
column_names = []

# Apply find_all() function with `th` element on first_launch_table
# Iterate each th element and apply the provided extract_column_from_header() to get a column name
# Append the Non-empty column name (`if name is not None and len(name) > 0`) into a list called column_names


Check the extracted column names


In [20]:
print(column_names)

[]


## TASK 3: Create a data frame by parsing the launch HTML tables


We will create an empty dictionary with keys from the extracted column names in the previous task. Later, this dictionary will be converted into a Pandas dataframe


launch_dict= dict.fromkeys(column_names)

# Remove an irrelvant column
del launch_dict['Date and time ( )']

# Let's initial the launch_dict with each value to be an empty list
launch_dict['Flight No.'] = []
launch_dict['Launch site'] = []
launch_dict['Payload'] = []
launch_dict['Payload mass'] = []
launch_dict['Orbit'] = []
launch_dict['Customer'] = []
launch_dict['Launch outcome'] = []
# Added some new columns
launch_dict['Version Booster']=[]
launch_dict['Booster landing']=[]
launch_dict['Date']=[]
launch_dict['Time']=[]

In [47]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# 1. Fetch Wikipedia page with error handling
try:
    url = "https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches"
    headers = {'User-Agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=headers, timeout=10)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, 'html.parser')
except Exception as e:
    print(f"Error fetching data: {e}")
    raise

# 2. Find launch table
tables = soup.find_all('table', class_='wikitable')
if len(tables) < 3:
    raise ValueError("Launch table not found")
table = tables[2]  # 3rd table contains launch data

# 3. Initialize dictionary with cleaned column names
launch_data = {
    'Flight No.': [],
    'Date': [],
    'Time (UTC)': [],
    'Booster Version': [],
    'Launch Site': [],
    'Payload': [],
    'Payload Mass (kg)': [],
    'Orbit': [],
    'Customer': [],
    'Launch Outcome': [],
    'Landing Result': []  # Renamed for clarity
}

# 4. Parse rows with robust error handling
for row in table.find_all('tr')[1:]:
    cells = row.find_all(['th', 'td'])
    if len(cells) < 10:
        continue
    
    try:
        # Flight Number
        launch_data['Flight No.'].append(cells[0].get_text(strip=True))
        
        # Date/Time (cleaned)
        datetime_str = cells[1].get_text(strip=True).replace(',', '')
        date_time = datetime_str.split()
        launch_data['Date'].append(date_time[0] if date_time else None)
        launch_data['Time (UTC)'].append(date_time[1] if len(date_time) > 1 else None)
        
        # Booster Version
        launch_data['Booster Version'].append(cells[2].get_text(strip=True))
        
        # Launch Site
        launch_data['Launch Site'].append(cells[3].get_text(strip=True))
        
        # Payload
        launch_data['Payload'].append(cells[4].get_text(strip=True))
        
        # Payload Mass (convert to clean integer)
        mass_str = cells[5].get_text(strip=True)
        mass = ''.join(filter(str.isdigit, mass_str)) if mass_str else None
        launch_data['Payload Mass (kg)'].append(int(mass) if mass else None)
        
        # Orbit
        launch_data['Orbit'].append(cells[6].get_text(strip=True))
        
        # Customer
        launch_data['Customer'].append(cells[7].get_text(strip=True))
        
        # Launch Outcome
        launch_data['Launch Outcome'].append(cells[8].get_text(strip=True))
        
        # Landing Result (extract text between parentheses)
        landing = cells[9].get_text(strip=True)
        launch_data['Landing Result'].append(
            landing.split('(')[-1].split(')')[0] if '(' in landing else landing
        )
    except Exception as e:
        print(f"Skipping row due to error: {e}")
        continue

# 5. Create DataFrame with proper types
df = pd.DataFrame(launch_data)
df['Flight No.'] = pd.to_numeric(df['Flight No.'], errors='coerce')
df['Payload Mass (kg)'] = pd.to_numeric(df['Payload Mass (kg)'], errors='coerce')

# 6. Final cleaned output
print(f"\nSuccess! Scraped {len(df)} launches:")
print(df.head().to_string())  # to_string() prevents truncation

# Optional: Save to CSV
df.to_csv('spacex_launches_clean.csv', index=False)


Success! Scraped 74 launches:
   Flight No.     Date Time (UTC) Booster Version            Launch Site                                 Payload  Payload Mass (kg) Orbit Customer Launch Outcome Landing Result
0         418  January          4    F9B5B1073‑20  Cape Canaveral,SLC‑40                           Thuraya 4-NGS       5.000110e+08   GTO  Thuraya        Success           ASOG
1         419  January          6    F9B5B1077‑17  Cape Canaveral,SLC‑40       Starlink:Group 6‑71(24satellites)       1.750039e+09   LEO   SpaceX        Success           JRTI
2         420  January          8     F9B5B1086‑3         Kennedy,LC‑39A     Starlink:Group 12-11(21 satellites)       1.650036e+09   LEO   SpaceX        Success           ASOG
3         421  January         10    F9B5B1071‑22      Vandenberg,SLC‑4E  NROL-153 (22Starshieldsatellites)[461]                NaN   LEO      NRO        Success         OCISLY
4         422  January         10    F9B5B1067‑25  Cape Canaveral,SLC‑40     Starlin

In [48]:
# Initialize launch dictionary with required columns
required_columns = [
    'Flight No.',
    'Date',
    'Time',
    'Version Booster',
    'Launch site',
    'Payload',
    'Payload mass',
    'Orbit',
    'Customer',
    'Launch outcome',
    'Booster landing'
]

# Create dictionary with empty lists
launch_dict = {}
for col in required_columns:
    launch_dict[col] = []

# Verify initialization
print("Successfully created launch_dict with columns:")
print(list(launch_dict.keys()))


Successfully created launch_dict with columns:
['Flight No.', 'Date', 'Time', 'Version Booster', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome', 'Booster landing']


Next, we just need to fill up the `launch_dict` with launch records extracted from table rows.


Usually, HTML tables in Wiki pages are likely to contain unexpected annotations and other types of noises, such as reference links `B0004.1[8]`, missing values `N/A [e]`, inconsistent formatting, etc.


To simplify the parsing process, we have provided an incomplete code snippet below to help you to fill up the `launch_dict`. Please complete the following code snippet with TODOs or you can choose to write your own logic to parse all launch tables:


After you have fill in the parsed launch record values into `launch_dict`, you can create a dataframe from it.


df= pd.DataFrame({ key:pd.Series(value) for key, value in launch_dict.items() })

We can now export it to a <b>CSV</b> for the next section, but to make the answers consistent and in case you have difficulties finishing this lab. 

Following labs will be using a provided dataset to make each lab independent. 


<code>df.to_csv('spacex_web_scraped.csv', index=False)</code>


In [58]:
# Export to CSV with verification
df.to_csv('spacex_web_scraped.csv', index=False)
print(f"✅ Successfully exported {len(df)} rows to spacex_web_scraped.csv")

# Optional: Add file check
import os
if os.path.exists('spacex_web_scraped.csv'):
    print(f"📄 File size: {os.path.getsize('spacex_web_scraped.csv')} bytes")
else:
    print("❌ Error: File not created")

✅ Successfully exported 74 rows to spacex_web_scraped.csv
📄 File size: 8869 bytes


## Authors


<a href="https://www.linkedin.com/in/yan-luo-96288783/">Yan Luo</a>


<a href="https://www.linkedin.com/in/nayefaboutayoun/">Nayef Abou Tayoun</a>


<!--
## Change Log
-->


<!--
| Date (YYYY-MM-DD) | Version | Changed By | Change Description      |
| ----------------- | ------- | ---------- | ----------------------- |
| 2021-06-09        | 1.0     | Yan Luo    | Tasks updates           |
| 2020-11-10        | 1.0     | Nayef      | Created the initial version |
-->


Copyright © 2021 IBM Corporation. All rights reserved.
